In [1]:
from scipy.stats import chisquare
from scipy import stats
import urllib.request
import string
import requests
from bs4 import BeautifulSoup
import pandas as pd
import multiprocessing
import gensim
import sys
import spacy
nlp=spacy.load('en')
import seaborn as sns
from nltk.tokenize import WordPunctTokenizer
tknzr = WordPunctTokenizer()
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from scipy import spatial
from collections import Counter
import numpy as np
from collections import defaultdict
from pprint import pprint
from gensim import corpora
import re
import logging
import requests
import urllib 
import re
import sys
import zipfile
from glob import glob
import statsmodels.api as sm
import scipy
import scipy.stats as ss
from scipy.signal import savgol_filter
import random
# np.set_printoptions(precision=4)
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

import matplotlib.pyplot as plt
%matplotlib inline


from sklearn import metrics
from sklearn.cluster import KMeans
# from the demo
from sklearn.datasets import load_digits
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF
from sklearn.preprocessing import scale
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from nltk.corpus import stopwords
from nltk.corpus import names
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

In [2]:
def flushPrint(s):
    sys.stdout.write('\r')
    sys.stdout.write('%s' % s)
    sys.stdout.flush()

In [3]:
# 获取名字，性别数据列表
from nltk.corpus import names
male_names = [name for name in names.words('male.txt')]
female_names = [name for name in names.words('female.txt')]
names = male_names+female_names

In [4]:
with open('./stopword.txt','r') as f:
    stopword=f.readlines()
stopword=[i.split('\n')[0] for i in stopword]

In [5]:
files=glob('./gutenberg/corpus/*.txt')

In [6]:
def get_maintext_lines_gutenberg(raw_text):
    lines = raw_text.split("\n")
    start_book_i = 0
    end_book_i = len(lines)-1
    # pass 1, this format is easy and gets 78.9% of books
    start1="START OF THIS PROJECT GUTENBERG EBOOK"
    start2="START OF THE PROJECT GUTENBERG EBOOK"
    end1="END OF THIS PROJECT GUTENBERG EBOOK"
    end2="END OF THE PROJECT GUTENBERG EBOOK"
    end3="END OF PROJECT GUTENBERG"
    for j,line in enumerate(lines):
        if (start1 in line) or (start2 in line):
            # and "***" in line and start_book[i] == 0 and j<.25*len(lines):
            start_book_i = j
        end_in_line = end1 in line or end2 in line or end3 in line.upper()
        if end_in_line and (end_book_i == (len(lines)-1)):
            #  and "***" in line and j>.75*len(lines)
            end_book_i = j
    # pass 2, this will bring us to 99%
    if (start_book_i == 0) and (end_book_i == len(lines)-1):
        for j,line in enumerate(lines):
            if ("end" in line.lower() or "****" in line) and  "small print" in line.lower() and j<.5*len(lines):
                start_book_i = j
            if "end" in line.lower() and "project gutenberg" in line.lower() and j>.75*len(lines):
                end_book_i = j
        # pass three, caught them all (check)
        if end_book_i == len(lines)-1:
            for j,line in enumerate(lines):
                if "THE END" in line and j>.9*len(lines):
                    end_book_i = j
    return lines[(start_book_i+1):(end_book_i)]

In [7]:
import json
with open("diction.txt","r") as f:
    ediction = json.loads(f.readline())

In [8]:
def chunkify(lines):
    # put them back together...
    raw_text = "\n".join(lines)
    # remove extra whitespace
    raw_text_1 = re.sub("\n[\\s]+?\n","\n\n",raw_text)
    # remove singleton newlines
    raw_text_2 = re.sub(r"([^\n])\n([^\n])",r"\1 \2",raw_text_1)
    raw_text_3 = raw_text_2.rstrip().lstrip()

    # three levels of significance
    # single newlines were already discarded (insignificant)
    # double newlines are real line breaks
    # triple newlines (or more) separate content

    # split on those triples (or more)
    big_chunks = re.split("\n\n\n+",raw_text_3)

    # now break them into the paragraphs
    small_chunks = list(map(lambda x: re.split("\n\n",x),big_chunks))
    combined_chunks = []
    [combined_chunks.extend(el) for el in small_chunks]
#     combined_chunks = []

#     for i in range(len(small_chunks)):
#         for j in range(len(small_chunks[i])):
#             combined_chunks.append((i,j,small_chunks[i][j]))
    return combined_chunks
#     return small_chunks

### charcter

In [70]:
def main_character(lead):
#     lead=[i.title() for i in lead]
    film_name_list = (set(lead) & set(male_names)) |\
                    (set(lead) & set(female_names))
    film_script_wordlist_counter = Counter(lead)
    df = pd.DataFrame([[i, film_script_wordlist_counter[i]] for i in film_name_list]\
                                      , columns=['name', 'count'])
    df['gender']=df['name'].apply(lambda x:'male' if x in male_names else 'female')
    df=df.sort_values(by='count', ascending=False)
    lead=df[df['gender']=='male']['name'].values[0]
    lead_n=df[df['gender']=='male']['count'].values[0]
    lead_gender=df['gender'].values[0]
    lead1=df[df['gender']=='female']['name'].values[0]
    lead1_n=df[df['gender']=='female']['count'].values[0]
    return lead,lead_n,lead1,lead1_n,lead_gender

In [104]:
characters=[]
for m,n in enumerate(files[:10]):
    flushPrint(m) 
    try:
        f = open(n,"r",encoding='ISO-8859-1')
        rawtext = f.read()
        f.close()
        lines = get_maintext_lines_gutenberg(rawtext)
        chunked = chunkify(lines)
        text=tknzr.tokenize(rawtext)
        characters.append([n.split('/')[-1].split('.')[0],main_character(text)\
                          ,len(text),len(chunked)])
    except Exception as e:
        print(e,n)
        pass

9

In [94]:
data=pd.DataFrame()

In [106]:
data['id']=[i[0] for i in characters]
data['name']=[i[1] for i in characters]
data['word']=[i[2] for i in characters]
data['para']=[i[3] for i in characters]

In [109]:
data.to_csv('./all_books.csv',index=False)

### all_books

In [273]:
book=pd.read_csv('./all_books.csv')
book['no']=book['id'].apply(lambda x: x.split('-')[0])
book=book.drop(['id'],axis=1)
book=book.drop_duplicates()
book['name']=book['name'].apply(lambda x: eval(x))
book=book[book['no']!='0']
book['character']=book['name'].apply(lambda x: [x[0],x[2],x[-1]])
book['lead_gender']=book['name'].apply(lambda x: x[-1])
book['name_c']=book['name'].apply(lambda x: x[1])
book['name_c1']=book['name'].apply(lambda x: x[3])

In [274]:
metadata=pd.read_csv('./gutenberg/metadata.csv')
metadata=metadata[metadata['lang']=='English']
metadata['class']=metadata['loc_class'].apply(lambda x: [i[:2] for i in x.split('|') if i[0] in [str(chr(i)) for i in range(65,91)]])
metadata=metadata.drop_duplicates(subset=['no'])

In [280]:
all_book=pd.merge(metadata,book,on=['no'])
all_book=all_book.drop_duplicates(subset=['no'])

In [264]:
choose=all_book[(all_book['name_c']>10)&(all_book['name_c1']>10)]

In [265]:
len(choose)

29871

In [238]:
def word_to_emotion(data):
    d=[]
    for word in tknzr.tokenize(data):
        try:
            if (word.lower() not in stopword):#&(word not in names):
                sim = ediction[word]
                if (sim<-3)|(sim>3):
                    d.append(sim)
        except:
            pass
    return np.mean(d)

In [233]:
characters=[]
for m,n in enumerate(files):
    flushPrint(m)
    try:
        f = open(n,"r",encoding='ISO-8859-1')
        rawtext = f.read()
        f.close()
        index=n.split('/')[-1].split('.')[0].split('-')[0]
        lines = get_maintext_lines_gutenberg(rawtext)
        chunked = chunkify(lines)
        lead=choose[choose['no']==index]['character'].values[0][0]
        lead1=choose[choose['no']==index]['character'].values[0][1]
        tp=[i for i,j in enumerate(chunked) if (lead in tknzr.tokenize(j)) & (lead1 in tknzr.tokenize(j))]
        m=[i for i,j in enumerate(chunked) if lead in tknzr.tokenize(j)]
        f=[i for i,j in enumerate(chunked) if lead1 in tknzr.tokenize(j)]
        w=[word_to_emotion(i) for i in chunked]
        characters.append([index,tp,m,f,w])
    except Exception as e:
        print(e,n)
        pass

1

/Users/xuhuimin/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/xuhuimin/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


3index 0 is out of bounds for axis 0 with size 0 ./gutenberg/corpus/18520.txt
8index 0 is out of bounds for axis 0 with size 0 ./gutenberg/corpus/91-0.txt
9index 0 is out of bounds for axis 0 with size 0 ./gutenberg/corpus/7353.txt
10index 0 is out of bounds for axis 0 with size 0 ./gutenberg/corpus/24436.txt
12index 0 is out of bounds for axis 0 with size 0 ./gutenberg/corpus/2607.txt
19index 0 is out of bounds for axis 0 with size 0 ./gutenberg/corpus/8920.txt
21index 0 is out of bounds for axis 0 with size 0 ./gutenberg/corpus/10966.txt
23index 0 is out of bounds for axis 0 with size 0 ./gutenberg/corpus/36-0.txt
25index 0 is out of bounds for axis 0 with size 0 ./gutenberg/corpus/21775.txt
26index 0 is out of bounds for axis 0 with size 0 ./gutenberg/corpus/847.txt
31index 0 is out of bounds for axis 0 with size 0 ./gutenberg/corpus/4272.txt
32index 0 is out of bounds for axis 0 with size 0 ./gutenberg/corpus/116-readme.txt
34index 0 is out of bounds for axis 0 with size 0 ./gutenb

In [234]:
data=pd.DataFrame()

In [240]:
data['index']=[i[0] for i in characters]
data['tp']=[i[1] for i in characters]
data['f']=[i[2] for i in characters]
data['m']=[i[3] for i in characters]
data['w']=[i[4] for i in characters]

In [267]:
data.to_csv('./all_books_emotion.csv',index=False)

In [268]:
index=[i[0] for i in characters]
tp=[i[1] for i in characters]
m=[i[2] for i in characters]
f=[i[3] for i in characters]
w=[i[4] for i in characters]

In [269]:
diction1=dict(zip(index,tp))
diction2=dict(zip(index,m))
diction3=dict(zip(index,f))
diction4=dict(zip(index,w))

In [270]:
choose['tp']=choose['no'].apply(lambda x: diction1[x] if x in diction1 else np.nan)
choose['m']=choose['no'].apply(lambda x: diction2[x] if x in diction2 else np.nan)
choose['f']=choose['no'].apply(lambda x: diction3[x] if x in diction3 else np.nan)
choose['w']=choose['no'].apply(lambda x: diction4[x] if x in diction4 else np.nan)

/Users/xuhuimin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/xuhuimin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/xuhuimin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in 

In [272]:
choose.to_csv('./choose.csv',index=False)